<a href="https://colab.research.google.com/github/jiw3026/SML/blob/main/Ensemble_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Ensemble Learning & RandomForest 진행

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
# 데이터 불러오기
train_path = 'https://raw.githubusercontent.com/jiw3026/SML/main/train.csv'
train = pd.read_csv(train_path, index_col = 'id')

test_path = 'https://raw.githubusercontent.com/jiw3026/SML/main/test.csv'
test = pd.read_csv(test_path, index_col = 'id')

In [ ]:
train.head()

,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
id,,,,,,,,,,
3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49.0
6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0
8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57.0
9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431.0


In [ ]:
test.head()

,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5
id,,,,,,,,,
0,7,20.7,0.0,1.3,62.0,954.0,0.041,44.0,27.0
1,17,30.0,0.0,5.4,33.0,1590.0,0.061,49.0,36.0
2,13,19.0,1.0,2.1,95.0,193.0,0.020,36.0,28.0
4,6,22.5,0.0,2.5,60.0,1185.0,0.027,52.0,38.0
5,22,14.6,1.0,3.4,93.0,218.0,0.041,18.0,15.0


In [ ]:
# 데이터 결측치 제거
test_df = test.copy()
train_df = train.copy()

train_df = train_df.dropna()

In [ ]:
# X값 y값(target) 분리
X = train_df.iloc[:, :-1]
y = train_df.iloc[:, -1]

In [ ]:
# train data, test data 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

####Bagging ensembles

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42), n_estimators=500,
    max_samples=100, bootstrap=True, random_state=42)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.02108433734939759


In [ ]:
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)
y_pred_tree = tree_clf.predict(X_test)
print(accuracy_score(y_test, y_pred_tree))

0.024096385542168676


#### Random Forests

In [ ]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(splitter="random", max_leaf_nodes=16, random_state=42),
    n_estimators=500, max_samples=1.0, bootstrap=True, random_state=42)

In [ ]:
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, random_state=42)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [ ]:
print(accuracy_score(y_test, y_pred_rf)) # Bagging ensembles이 더 높은 accuracy

0.015060240963855422


In [ ]:
rnd_clf.feature_importances_ #hour의 중요도가 가장 큼을 확인

array([0.19197414, 0.15893069, 0.01580561, 0.10144255, 0.10226012,
       0.09111403, 0.10184506, 0.10586844, 0.13075936])

#### RMSE 비교
- RMSE란? ( Root Mean Squared Error )

  표준편차와 동일
  
  특정 수치에 대한 예측의 정확도를 표현할 때, Accuracy로 판단하기에는 정확도를 올바르게 표기할 수 없어, RMSE 수치로 정확도 판단
  
  일반적으로 해당 수치가 낮을수록 정확도가 높다고 판단

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
# Bagging ensembles RMSE
RMSE_bag = mean_squared_error(y_test, y_pred)**0.5
print(RMSE_bag)

69.54237245208896


In [ ]:
# RandomForest ensembles RMSE
RMSE_rf = mean_squared_error(y_test, y_pred_rf)**0.5
print(RMSE_rf)

# RMSE값도 Bagging emsembles이 더 낮게 나옴

75.25977101034869
